<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BA/blob/main/cl_km_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://github.com/cristiandarioortegayubro/BA/blob/main/dba.png?raw=true)

![](https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png)

## **Carga de bibliotecas necesarias**

### **Para el tratamiento de los datos**

In [1]:
import pandas as pd
import numpy as np

### **Para gráficos**

In [2]:
import plotly.express as px
import plotly.graph_objects as go

### **Para preprocesamiento de datos y modelo**

In [3]:
import sklearn
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import sklearn.metrics as metrics
from sklearn.metrics import silhouette_score

## **Extracción de Datos - Creación del DataFrame**

In [4]:
url = "https://raw.githubusercontent.com/LucaAPiattelli/Diplomatura_Business_Analytics_UDA/main/Modulo_11_Agrupacion/"

In [5]:
archivo = "clientes01.csv"

In [6]:
clientes = pd.read_csv(url+archivo)
clientes

,ID,Trabajo,Edad,Salario,Compra
0,0,1,19,19000,No
1,1,1,35,20000,No
2,2,0,26,43000,No
3,3,0,27,57000,No
4,4,1,19,76000,No
...,...,...,...,...,...
395,395,0,46,41000,Si
396,396,1,51,23000,Si
397,397,0,50,20000,Si
398,398,1,36,33000,No


El dataframe contiene 5 variables y 400 observaciones. 

Las variables indican:
- **ID:** La identificación del cliente
- **Trabajo:** Corresponde 1 cuando es dependiente y 0 cuando es independiente
- **Edad:** La edad del cliente
- **Salario:** El salario estimado del cliente
- **Compra:** Si, cuando el cliente ha comprado y no cuando no lo ha hecho.

Pero para el desarrollo del algoritmo se simplificara a un problema bidimensional, seleccionando las variables de edad y salario.

## **Eliminando variables**

In [7]:
clientes.info() #visualizacion de los tipos de datos del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       400 non-null    int64 
 1   Trabajo  400 non-null    int64 
 2   Edad     400 non-null    int64 
 3   Salario  400 non-null    int64 
 4   Compra   400 non-null    object
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [8]:
clientes = clientes.drop(columns=["ID","Compra"]) #se elimina la variable no relevante
clientes #vista del dataframe

,Trabajo,Edad,Salario
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000
...,...,...,...
395,0,46,41000
396,1,51,23000
397,0,50,20000
398,1,36,33000


## **Número de clusters**

In [9]:
clusters = pd.DataFrame()
inertia = []

In [10]:
clusters["cluster_range"] = range(1, 10)

In [11]:
for k in clusters["cluster_range"]:
    kmeans = cluster.KMeans(n_clusters=k, random_state=8).fit(clientes)
    inertia.append(kmeans.inertia_)

In [12]:
clusters["inertia"] = inertia

In [13]:
clusters.inertia = round(clusters.inertia, 4)

In [14]:
clusters.head(10)

,cluster_range,inertia
0,1,4.638785e+11
1,2,1.651974e+11
2,3,5.952127e+10
3,4,3.349234e+10
4,5,1.998340e+10
5,6,1.468427e+10
6,7,1.012251e+10
7,8,8.096856e+09
8,9,6.340387e+09


### Graficando clusters óptimos.

In [15]:
fig = px.line(clusters,
              x = "cluster_range",
              y = "inertia",
              markers = True,
              title = "Metodo del codo",
              template = "gridon",
              labels = {"cluster_range":"clusters"})
fig.show()

# **Evaluando el Algoritmo**

## **Algoritmo K-means**

In [16]:
clientes.head()

,Trabajo,Edad,Salario
0,1,19,19000
1,1,35,20000
2,0,26,43000
3,0,27,57000
4,1,19,76000


In [19]:
km = cluster.KMeans(n_clusters = 4, n_init = 20, random_state = 123)

In [20]:
km.fit(clientes)

KMeans(n_clusters=4, n_init=20, random_state=123)

In [21]:
centroids = km.cluster_centers_
labels = km.labels_

In [22]:
centroids

array([[5.18518519e-01, 3.68888889e+01, 5.37407407e+04],
       [3.80281690e-01, 4.24788732e+01, 1.26338028e+05],
       [5.29850746e-01, 3.59701493e+01, 8.04850746e+04],
       [4.82758621e-01, 3.72643678e+01, 2.68735632e+04]])

In [24]:
centroids = pd.DataFrame(centroids, columns=["Trabajo", 'Edad', 'Salario'])
centroids

,Trabajo,Edad,Salario
0,0.518519,36.888889,53740.740741
1,0.380282,42.478873,126338.028169
2,0.529851,35.970149,80485.074627
3,0.482759,37.264368,26873.563218


In [25]:
labels

array([3, 3, 0, 0, 2, 0, 2, 1, 3, 0, 2, 0, 2, 3, 2, 2, 3, 3, 3, 3, 3, 0,
       0, 3, 3, 3, 3, 3, 0, 3, 2, 1, 3, 0, 2, 3, 3, 0, 2, 3, 3, 0, 1, 3,
       2, 3, 2, 0, 1, 2, 3, 0, 2, 3, 0, 0, 0, 2, 3, 1, 3, 2, 0, 1, 2, 0,
       3, 2, 0, 2, 2, 3, 3, 1, 3, 1, 0, 3, 2, 3, 2, 0, 0, 2, 0, 1, 0, 2,
       2, 0, 2, 1, 3, 3, 2, 0, 3, 1, 2, 3, 2, 0, 2, 1, 3, 2, 3, 2, 2, 2,
       2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 3, 3, 2, 0, 3,
       2, 2, 0, 0, 2, 1, 0, 3, 2, 2, 0, 2, 3, 2, 2, 3, 0, 2, 3, 0, 2, 0,
       0, 3, 0, 2, 3, 1, 2, 2, 3, 3, 2, 2, 0, 2, 1, 0, 2, 1, 1, 0, 2, 3,
       0, 3, 3, 3, 3, 2, 1, 0, 0, 0, 2, 0, 2, 3, 2, 3, 0, 2, 2, 0, 2, 3,
       2, 3, 3, 2, 1, 2, 2, 0, 1, 1, 1, 3, 2, 1, 0, 0, 0, 1, 0, 2, 2, 1,
       2, 2, 1, 2, 0, 0, 1, 1, 2, 2, 1, 0, 1, 2, 1, 2, 0, 2, 2, 1, 1, 0,
       2, 1, 2, 1, 0, 1, 0, 2, 3, 0, 1, 1, 0, 2, 2, 0, 2, 1, 2, 1, 1, 2,
       2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 3, 2, 2, 2, 3, 3, 2, 0, 2, 3, 1, 2,
       0, 1, 2, 2, 1, 2, 3, 2, 0, 0, 2, 1, 2, 1, 3,

## **Métricas**

### **Calinski Harabasz**

In [26]:
metrics.calinski_harabasz_score(clientes, labels)

1698.0186406648688

### **Silhouette**

In [27]:
metrics.silhouette_score(clientes, labels)

0.6065988712060131

### **Davies Bouldin**

In [28]:
metrics.davies_bouldin_score(clientes, labels)

0.46056050670794835

In [29]:
clientes['cluster'] = labels

In [30]:
clientes.head()

,Trabajo,Edad,Salario,cluster
0,1,19,19000,3
1,1,35,20000,3
2,0,26,43000,0
3,0,27,57000,0
4,1,19,76000,2


## **Grafico**

### **Plotly**

In [36]:
fig = go.Figure([go.Scatter(x = clientes.Edad, 
                            y = clientes.Salario,
                            mode = "markers",
                            name = "Clusters",
                            marker = dict(color = clientes.Salario,
                                          colorscale = 'bluered',
                                          showscale = False)),

                 go.Scatter(x = centroids.Edad,
                            y = centroids.Salario,
                            mode = "markers",
                            name = "Centroide",
                            marker_color = "orange",
                            marker = dict(size = 12)),
                 ])

fig.update_layout(template =    "gridon",
                  title =       "Edad y Salarios",
                  yaxis_title = "Y",
                  xaxis_title = "X")

fig.show()